# Test Agent on PushBlock Environment

## Step 1: Import the libraries

In [ ]:
import time
import random
import math
from collections import deque

import numpy as np
from IPython.display import clear_output
import matplotlib.pyplot as plt
import torch

from gym_unity.envs import UnityEnv

In [ ]:
from ddqn_agent import DDQNAgent
from model import DDQNLinear

## Step 2: Create our environment

Initialize the environment in the code cell below.


In [ ]:
ENV_PATH = '../unity_envs/PushBlock/'
ENV_NAME = 'Unity Environment'
env = UnityEnv(ENV_PATH + ENV_NAME, worker_id=0, multiagent=True)

In [ ]:
# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device: ", device)

## Step 3: Creating our Agent

In [ ]:
STATE_SIZE = env.observation_space.shape[0]
ACTION_SIZE = 7
GAMMA = 0.99                # discount factor
BUFFER_SIZE = int(1e5)      # replay buffer size
BATCH_SIZE = 256            # Update batch size
LR = 1e-3                   # learning rate 
TAU = 1e-2                  # for soft update of target parameters
UPDATE_EVERY = 5            # how often to update the network 

agent = DDQNAgent(STATE_SIZE, ACTION_SIZE, BUFFER_SIZE, BATCH_SIZE, GAMMA, LR, TAU, UPDATE_EVERY, device)
scores = agent.load_model("pushblock_solved.pth")

In [ ]:
def plot_result(scores):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    plt.plot(np.arange(len(scores)), scores)
    plt.ylabel('Score')
    plt.xlabel('Episode #')
    plt.show()

## Step 4: Testing the Agent

In [ ]:
test_scores = []
for episode in range(100):
    episode_rewards = np.zeros(32)
    states = np.array(env.reset())
    while True:
        actions = agent.act(states)
        next_states, rewards, dones, _ = env.step(actions.tolist())
        episode_rewards += rewards
        if np.any(dones):
            break
        else:
            states = np.array(next_states)
    test_scores.append(np.mean(episode_rewards))
    clear_output(wait=True)
    plot_result(test_scores)
env.close()